# Démonstration : Authenticated Encryption (AEAD)

**Objectifs** :
- Comprendre pourquoi le chiffrement seul ne suffit pas
- Implémenter AES-GCM (standard industriel)
- Comparer avec ChaCha20-Poly1305
- Démontrer l'importance de l'authentification

In [ ]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM, ChaCha20Poly1305
import secrets
import time

## 1. Problème : Chiffrement sans intégrité

Le chiffrement CTR seul permet la **malléabilité** : un attaquant peut modifier le ciphertext de manière contrôlée !

In [ ]:
# Démonstration de malléabilité (CTR mode)
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def demonstrate_malleability():
    """
    Montre qu'un attaquant peut modifier un montant dans un ciphertext CTR.
    """
    key = secrets.token_bytes(16)
    nonce = secrets.token_bytes(16)
    
    # Alice chiffre : "TRANSFER $1000"
    message = b"TRANSFER $1000"
    
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()
    
    print(f"Message original : {message}")
    print(f"Ciphertext (hex) : {ciphertext.hex()}")
    
    # Eve intercepte et modifie (sans connaître la clé !)
    # Elle veut changer "$1000" en "$9999"
    # Position du '1' dans "$1000" : index 10
    
    # XOR nécessaire : '1' XOR '9' = 0x31 XOR 0x39 = 0x08
    modified_ciphertext = bytearray(ciphertext)
    modified_ciphertext[10] ^= ord('1') ^ ord('9')  # 1→9
    modified_ciphertext[11] ^= ord('0') ^ ord('9')  # 0→9
    modified_ciphertext[12] ^= ord('0') ^ ord('9')  # 0→9
    modified_ciphertext[13] ^= ord('0') ^ ord('9')  # 0→9
    
    # Bob déchiffre (ne détecte PAS la modification !)
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(bytes(modified_ciphertext)) + decryptor.finalize()
    
    print(f"\n🔓 Message déchiffré (modifié) : {decrypted}")
    print(f"\n❌ Bob ne peut PAS détecter la modification !")
    print(f"⚠️  C'est pourquoi on a BESOIN d'authentification !")

demonstrate_malleability()

## 2. Solution : AEAD (Authenticated Encryption with Associated Data)

**AEAD** combine :
- **Confidentialité** : Chiffrement du message
- **Intégrité/Authenticité** : TAG qui détecte toute modification
- **Associated Data** : Données non chiffrées mais authentifiées

## 3. AES-GCM (Galois/Counter Mode)

**Le standard industriel**
- Chiffrement : AES-CTR
- Authentification : GMAC (multiplication dans GF(2^128))
- Très rapide avec accélération matérielle (AES-NI)
- Utilisé dans TLS 1.3, IPsec, SSH

In [ ]:
# Configuration AES-GCM
key_gcm = AESGCM.generate_key(bit_length=256)  # AES-256
aesgcm = AESGCM(key_gcm)

print(f"Clé AES-GCM (hex) : {key_gcm.hex()}")
print(f"Taille : {len(key_gcm)} bytes = {len(key_gcm)*8} bits")

In [ ]:
def aes_gcm_demo():
    """
    Démonstration complète d'AES-GCM.
    """
    # Message et données associées
    message = b"Secret payment: $1000"
    associated_data = b"To: Bob, From: Alice, Date: 2026-01-11"  # Non chiffré mais authentifié
    
    # Nonce (DOIT être unique pour chaque message avec la même clé !)
    nonce = secrets.token_bytes(12)  # 96 bits recommandé pour GCM
    
    print("=" * 60)
    print("CHIFFREMENT AES-GCM")
    print("=" * 60)
    print(f"Message       : {message}")
    print(f"Assoc. Data   : {associated_data}")
    print(f"Nonce (hex)   : {nonce.hex()}")
    
    # Chiffrement + authentification
    ciphertext = aesgcm.encrypt(nonce, message, associated_data)
    
    print(f"\nCiphertext (hex) : {ciphertext.hex()}")
    print(f"Taille : {len(ciphertext)} bytes (message: {len(message)}, tag: 16)")
    
    # Déchiffrement + vérification
    try:
        plaintext = aesgcm.decrypt(nonce, ciphertext, associated_data)
        print(f"\n✅ Déchiffrement réussi : {plaintext}")
    except Exception as e:
        print(f"\n❌ Erreur de déchiffrement : {e}")
    
    # Test 1 : Modifier le ciphertext
    print("\n" + "=" * 60)
    print("TEST 1 : Modification du ciphertext")
    print("=" * 60)
    
    modified_ciphertext = bytearray(ciphertext)
    modified_ciphertext[0] ^= 0xFF  # Flipper des bits
    
    try:
        aesgcm.decrypt(nonce, bytes(modified_ciphertext), associated_data)
        print("❌ ERREUR : Modification non détectée !")
    except Exception:
        print("✅ Modification DÉTECTÉE : déchiffrement refusé")
    
    # Test 2 : Modifier les associated data
    print("\n" + "=" * 60)
    print("TEST 2 : Modification des associated data")
    print("=" * 60)
    
    wrong_ad = b"To: Eve, From: Alice, Date: 2026-01-11"  # Eve change le destinataire !
    
    try:
        aesgcm.decrypt(nonce, ciphertext, wrong_ad)
        print("❌ ERREUR : Modification AD non détectée !")
    except Exception:
        print("✅ Modification AD DÉTECTÉE : déchiffrement refusé")
    
    # Test 3 : Réutilisation de nonce (CATASTROPHE)
    print("\n" + "=" * 60)
    print("TEST 3 : Réutilisation de nonce (DANGER !)")
    print("=" * 60)
    
    message2 = b"Another secret message"
    ciphertext2 = aesgcm.encrypt(nonce, message2, associated_data)  # MÊME nonce !
    
    print("⚠️  Même nonce réutilisé pour deux messages différents")
    print("⚠️  Ceci permet de retrouver la clé d'authentification !")
    print("⚠️  JAMAIS réutiliser un nonce avec la même clé en AES-GCM")

aes_gcm_demo()

## 4. ChaCha20-Poly1305

**Alternative moderne à AES-GCM**
- Chiffrement : ChaCha20 (stream cipher)
- Authentification : Poly1305 (MAC)
- Plus rapide qu'AES-GCM sans accélération matérielle
- Résistant aux timing attacks
- Utilisé dans TLS 1.3, WireGuard VPN

In [ ]:
# Configuration ChaCha20-Poly1305
key_chacha = ChaCha20Poly1305.generate_key()
chacha = ChaCha20Poly1305(key_chacha)

print(f"Clé ChaCha20-Poly1305 (hex) : {key_chacha.hex()}")
print(f"Taille : {len(key_chacha)} bytes = {len(key_chacha)*8} bits")

In [ ]:
def chacha_poly1305_demo():
    """
    Démonstration ChaCha20-Poly1305.
    """
    message = b"Top secret data encrypted with ChaCha20"
    associated_data = b"Metadata: timestamp, sender, etc."
    nonce = secrets.token_bytes(12)  # 96 bits
    
    print("=" * 60)
    print("CHIFFREMENT ChaCha20-Poly1305")
    print("=" * 60)
    print(f"Message     : {message}")
    print(f"Assoc. Data : {associated_data}")
    
    # Chiffrement
    ciphertext = chacha.encrypt(nonce, message, associated_data)
    print(f"\nCiphertext (hex) : {ciphertext.hex()}")
    
    # Déchiffrement
    plaintext = chacha.decrypt(nonce, ciphertext, associated_data)
    print(f"\n✅ Déchiffrement réussi : {plaintext}")
    
    # Test de modification
    print("\n" + "=" * 60)
    print("TEST : Modification du ciphertext")
    print("=" * 60)
    
    modified = bytearray(ciphertext)
    modified[0] ^= 0x01
    
    try:
        chacha.decrypt(nonce, bytes(modified), associated_data)
        print("❌ ERREUR : Modification non détectée !")
    except Exception:
        print("✅ Modification DÉTECTÉE et REJETÉE")

chacha_poly1305_demo()

## 5. Comparaison de performance

In [ ]:
def benchmark_aead():
    """
    Compare les performances d'AES-GCM vs ChaCha20-Poly1305.
    """
    # Données de test
    sizes = [1024, 10240, 102400, 1024000]  # 1KB, 10KB, 100KB, 1MB
    iterations = 1000
    
    print("=" * 70)
    print("BENCHMARK : AES-GCM vs ChaCha20-Poly1305")
    print("=" * 70)
    print(f"Itérations : {iterations} par taille\n")
    
    for size in sizes:
        data = secrets.token_bytes(size)
        ad = b"Associated data"
        
        # AES-GCM
        nonce_gcm = secrets.token_bytes(12)
        start = time.perf_counter()
        for _ in range(iterations):
            ct = aesgcm.encrypt(nonce_gcm, data, ad)
            aesgcm.decrypt(nonce_gcm, ct, ad)
        time_gcm = time.perf_counter() - start
        
        # ChaCha20-Poly1305
        nonce_chacha = secrets.token_bytes(12)
        start = time.perf_counter()
        for _ in range(iterations):
            ct = chacha.encrypt(nonce_chacha, data, ad)
            chacha.decrypt(nonce_chacha, ct, ad)
        time_chacha = time.perf_counter() - start
        
        throughput_gcm = (size * iterations * 2) / time_gcm / 1_000_000  # MB/s
        throughput_chacha = (size * iterations * 2) / time_chacha / 1_000_000
        
        print(f"Taille : {size:>7} bytes ({size//1024:>4} KB)")
        print(f"  AES-GCM          : {time_gcm:>6.3f}s ({throughput_gcm:>6.1f} MB/s)")
        print(f"  ChaCha20-Poly1305: {time_chacha:>6.3f}s ({throughput_chacha:>6.1f} MB/s)")
        print()
    
    print("Note : Les performances dépendent du CPU et de l'accélération matérielle AES-NI")

benchmark_aead()

## 6. Bonnes pratiques AEAD

### ✅ À FAIRE

1. **Nonce unique et imprévisible** pour chaque message
2. **Authentifier avant de déchiffrer** (AEAD le fait automatiquement)
3. **Utiliser associated data** pour métadonnées importantes
4. **Rotation des clés** régulière
5. **Choisir AES-GCM** si accélération matérielle disponible, **ChaCha20-Poly1305** sinon

### ❌ À ÉVITER

1. **JAMAIS réutiliser un nonce** avec la même clé
2. **Ne pas déchiffrer** si la vérification du tag échoue
3. **Ne pas utiliser** des nonces prévisibles/séquentiels sans compteur sécurisé
4. **Ne pas implémenter** sa propre AEAD (utiliser bibliothèques auditées)
5. **Ne pas ignorer** les erreurs de déchiffrement

### 🔐 Gestion des nonces

**Option 1** : Nonce aléatoire (recommandé si < 2^32 messages)
```python
nonce = secrets.token_bytes(12)  # 96 bits aléatoires
```

**Option 2** : Compteur sécurisé (pour grand volume)
```python
# Compteur atomique, jamais réinitialisé
nonce = counter.to_bytes(12, 'big')
counter += 1
```

**Option 3** : Hybrid (timestamp + random)
```python
timestamp = int(time.time()).to_bytes(4, 'big')
random_part = secrets.token_bytes(8)
nonce = timestamp + random_part
```

## Conclusion

**Points clés** :
- Le chiffrement seul NE PROTÈGE PAS contre les modifications
- AEAD garantit confidentialité + intégrité + authenticité
- AES-GCM : standard industriel (TLS, IPsec, SSH)
- ChaCha20-Poly1305 : alternative moderne, plus rapide sans AES-NI
- Nonce management est CRITIQUE

**En pratique** : Toujours utiliser AEAD (jamais chiffrement seul) !